In [1]:
# Production Plan Timeline Generator
# This script reads an Excel file, processes production data,
# and generates a visual timeline as a downloadable image.

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import io
from google.colab import files
import matplotlib.dates as mdates # Import for date formatting

def generate_timeline(df):
    """
    Processes the production plan data and generates a timeline.

    Args:
        df (pd.DataFrame): The DataFrame containing the production plan data.

    Returns:
        A plot object (matplotlib.pyplot) for the generated timeline.
    """
    # Define colors for each task
    colors = {
        'processing': 'darkgreen',
        'wash': 'purple', # Changed wash color to purple
        'changeover': 'darkorange'
    }

    # Initialize a list to hold all tasks for plotting
    tasks = []

    # Get the start date and time from the first row
    try:
        start_time_of_week = pd.to_datetime(df.loc[0, 'Date from'])
    except Exception as e:
        print(f"Error parsing 'Date from' column: {e}. Please ensure it's in a valid date/time format.")
        return

    current_time = start_time_of_week

    # Read wash duration and gap from the first row and convert to integers
    try:
        wash_duration_mins = int(df.loc[0, 'Duration'])
        wash_gap_mins = int(df.loc[0, 'Gap'])
        wash_duration = timedelta(minutes=wash_duration_mins)
        gap_duration = timedelta(minutes=wash_gap_mins)

    except KeyError as e:
        print(f"Warning: Missing wash column: {e}. Wash cycle will not be scheduled.")
        wash_duration = timedelta(minutes=0) # Set wash duration to 0 if columns are missing
        gap_duration = timedelta(hours=0) # Set wash gap to 0
    except ValueError as e:
         print(f"Warning: Error converting wash duration or gap to integer: {e}. Please ensure 'Duration' and 'Gap' columns contain valid numbers. Wash cycle will not be scheduled.")
         wash_duration = timedelta(minutes=0)
         gap_duration = timedelta(hours=0)
    except Exception as e:
        print(f"Warning: Error reading wash time: {e}. Wash cycle will not be scheduled.")
        wash_duration = timedelta(minutes=0)
        gap_duration = timedelta(hours=0)

    # Determine the start time for the first wash and initialize last_wash_end_time
    first_wash_time = None
    try:
        first_wash_time = pd.to_datetime(df.loc[0, 'First Wash Time'])
        last_wash_end_time = first_wash_time
    except KeyError:
        print("Warning: 'First Wash Time' column not found. Scheduling first wash based on 'Date from' and 'Gap'.")
        last_wash_end_time = start_time_of_week # Initialize last wash end time based on the start of the week if 'First Wash Time' is missing
    except Exception as e:
        print(f"Warning: Error reading 'First Wash Time': {e}. Scheduling first wash based on 'Date from' and 'Gap'.")
        last_wash_end_time = start_time_of_week # Initialize last wash end time based on the start of the week if 'First Wash Time' has an error

    # If a specific first wash time is provided, add it as a task
    if first_wash_time and wash_duration > timedelta(minutes=0):
         tasks.append({
            'start': first_wash_time,
            'end': first_wash_time + wash_duration,
            'duration_hours': wash_duration.total_seconds() / 3600,
            'task': 'wash',
            'product': 'Scheduled Wash',
            'order': -2 # Use a smaller order to place it before products
         })
         last_wash_end_time = first_wash_time + wash_duration # Update last_wash_end_time after scheduling the explicit first wash


    # Iterate through each row of the DataFrame to build the timeline for products
    for i, row in df.iterrows():
        product_name = row['product name']
        quantity_liters = row['quantity liters']
        process_speed = row['process speed per hour']
        line_efficiency = row['line efficiency']
        change_over_mins = row['Change Over']

        # Rule: Changeover time
        # The changeover for the current product happens at the start of its slot.
        if i > 0:
            change_over_duration = timedelta(minutes=change_over_mins)
            changeover_end_time = current_time + change_over_duration
            remaining_changeover_duration = change_over_duration

            # Schedule washes that fall within the changeover period
            next_wash_start_time = last_wash_end_time + gap_duration
            scheduled_washes_in_changeover = []
            while next_wash_start_time < changeover_end_time:
                 wash_end_time = next_wash_start_time + wash_duration
                 scheduled_washes_in_changeover.append({
                    'start': next_wash_start_time,
                    'end': wash_end_time
                 })
                 last_wash_end_time = wash_end_time
                 next_wash_start_time = last_wash_end_time + gap_duration

            # Check if changeover overlaps with any scheduled wash
            changeover_overlaps_with_wash = any(max(current_time, wash['start']) < min(changeover_end_time, wash['end']) for wash in scheduled_washes_in_changeover)

            if changeover_overlaps_with_wash:
                # If changeover overlaps with a wash, skip the changeover and move current_time to the end of the last overlapping wash
                overlapping_washes = [wash for wash in scheduled_washes_in_changeover if max(current_time, wash['start']) < min(changeover_end_time, wash['end'])]
                if overlapping_washes:
                    last_overlapping_wash_end = max(wash['end'] for wash in overlapping_washes)
                    current_time = last_overlapping_wash_end
                # Also add the scheduled wash tasks that fell within the original changeover period
                for wash in scheduled_washes_in_changeover:
                     # Avoid adding the initial explicit first wash again if it was already added
                     if 'order' not in wash or wash['order'] != -2:
                          tasks.append({
                            'start': wash['start'],
                            'end': wash['end'],
                            'duration_hours': (wash['end'] - wash['start']).total_seconds() / 3600,
                            'task': 'wash',
                            'product': 'Scheduled Wash',
                            'order': -1
                         })


            else:
                # No overlap, add changeover as planned
                tasks.append({
                    'start': current_time,
                    'end': changeover_end_time,
                    'duration_hours': change_over_mins / 60,
                    'task': 'changeover',
                    'product': product_name,
                    'order': i # Add order to maintain original sequence
                })
                current_time = changeover_end_time


        # Calculate effective processing speed and total processing time
        effective_speed = process_speed * line_efficiency
        total_processing_hours = quantity_liters / effective_speed
        processing_end_time = current_time + timedelta(hours=total_processing_hours)

        # Calculate total overlap time with washes during processing
        total_wash_overlap_duration = timedelta(minutes=0)
        next_wash_start_time = last_wash_end_time + gap_duration
        while next_wash_start_time < processing_end_time + total_wash_overlap_duration: # Check against potential extended end time
             wash_end_time = next_wash_start_time + wash_duration
             # Check for overlap with the current processing block (considering potential extension)
             overlap_start = max(current_time, next_wash_start_time)
             overlap_end = min(processing_end_time + total_wash_overlap_duration, wash_end_time)

             if overlap_start < overlap_end:
                 overlap_duration = overlap_end - overlap_start
                 total_wash_overlap_duration += overlap_duration

                 # Avoid adding the initial explicit first wash again if it was already added
                 if not (first_wash_time and next_wash_start_time == first_wash_time):
                     tasks.append({
                        'start': next_wash_start_time,
                        'end': wash_end_time,
                        'duration_hours': wash_duration.total_seconds() / 3600,
                        'task': 'wash',
                        'product': 'Scheduled Wash',
                        'order': -1
                     })
                 last_wash_end_time = wash_end_time
                 next_wash_start_time = last_wash_end_time + gap_duration
             else:
                # If the next wash doesn't overlap with the current processing block (even with extension),
                # break the loop to avoid infinite loop if gap is very large
                break


        # Extend processing end time by the total wash overlap duration
        extended_processing_end_time = processing_end_time + total_wash_overlap_duration

        # Add processing segments, breaking for washes
        segment_start_time = current_time
        # Filter washes that are within the original processing time range for segmenting
        scheduled_wash_intervals_for_segmenting = [(wash['start'], wash['end']) for wash in tasks if wash['task'] == 'wash' and max(current_time, wash['start']) < min(processing_end_time, wash['end'])]
        scheduled_wash_intervals_for_segmenting.sort()


        for wash_start, wash_end in scheduled_wash_intervals_for_segmenting:
            # Add processing segment before wash
            if segment_start_time < wash_start:
                tasks.append({
                   'start': segment_start_time,
                   'end': wash_start,
                   'duration_hours': (wash_start - segment_start_time).total_seconds() / 3600,
                   'task': 'processing',
                   'product': product_name,
                   'order': i
               })
            segment_start_time = max(segment_start_time, wash_end) # Start next segment after wash

        # Add remaining processing segment after any washes, extending to the adjusted end time
        if segment_start_time < extended_processing_end_time:
             tasks.append({
                'start': segment_start_time,
                'end': extended_processing_end_time,
                'duration_hours': (extended_processing_end_time - segment_start_time).total_seconds() / 3600,
                'task': 'processing',
                'product': product_name,
                'order': i
            })

        current_time = extended_processing_end_time # Update current_time based on the extended end time


    # Create the plot
    fig, ax = plt.subplots(figsize=(18, 8)) # Increased figure width
    ax.set_facecolor('white') # Set background color to white

    y_pos = 0
    product_y_positions = {}
    product_order = []

    # Create a DataFrame from tasks and sort by the 'order' column to maintain original sequence
    tasks_df = pd.DataFrame(tasks).sort_values(by='order')

    # Get unique products in the original order, placing 'Scheduled Wash' at the top
    unique_products_ordered = ['Scheduled Wash'] + [p for p in tasks_df['product'].unique() if p != 'Scheduled Wash']


    for product_name in unique_products_ordered:
        group = tasks_df[tasks_df['product'] == product_name].sort_values(by='start') # Sort tasks within product by start time
        product_y_positions[product_name] = y_pos
        product_order.append(product_name)
        for _, task in group.iterrows():
            ax.broken_barh([(task['start'], task['end'] - task['start'])], (y_pos - 0.4, 0.8),
                           facecolors=colors[task['task']], edgecolor='black')
            # Add vertical lines from the start and end of each task, extending to the bottom of the plot
            ax.vlines(task['start'], ymin=-0.5, ymax=y_pos + 0.4, color='gray', linestyle='-', linewidth=0.5, alpha=0.5) # Adjusted ymin and ymax
            ax.vlines(task['end'], ymin=-0.5, ymax=y_pos + 0.4, color='gray', linestyle='-', linewidth=0.5, alpha=0.5) # Adjusted ymin and ymax

        y_pos += 1

    # Set up the plot aesthetics
    ax.set_yticks(list(product_y_positions.values()))
    ax.set_yticklabels(product_order) # Use the ordered product names for y-axis labels
    ax.set_xlabel("Time")
    ax.set_title("Weekly Production Plan Timeline")
    ax.grid(True)
    ax.invert_yaxis()  # Products at the top (with wash at the very top)

    # Format x-axis to show day name and time, and add vertical lines for day divisions
    ax.xaxis.set_major_locator(mdates.DayLocator()) # Major ticks at the start of each day
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %m-%d')) # Format: AbbreviatedWeekday Month-Day
    ax.xaxis.set_minor_locator(mdates.HourLocator(interval=3)) # Minor ticks every 3 hours
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M')) # Format: Hour:Minute


    # Add vertical lines for day divisions
    if tasks:
        first_date = tasks_df['start'].min().floor('D')
        last_date = tasks_df['end'].max().ceil('D')
        delta_days = (last_date - first_date).days + 1

        for day in range(delta_days):
            day_start = first_date + timedelta(days=day)
            ax.axvline(day_start, color='gray', linestyle='--', linewidth=1, alpha=0.6) # Added alpha for clarity


    # Add time labels for each task start and end time
    if tasks:
        for _, task in tasks_df.iterrows():
            ax.text(task['start'], -0.1, task['start'].strftime('%H:%M'), rotation=90, va='top', ha='right', fontsize=8, fontweight='bold')
            ax.text(task['end'], -0.1, task['end'].strftime('%H:%M'), rotation=90, va='top', ha='right', fontsize=8, fontweight='bold')


    # Rotate date labels for better readability and move them outside
    plt.xticks(rotation=90, ha='right', va='top') # Adjusted alignment for major ticks
    plt.setp(ax.get_xminorticklabels(), rotation=90, ha='right', va='top') # Adjusted alignment for minor ticks

    # Add small vertical dotted lines at each major and minor tick
    ax.tick_params(axis='x', which='both', direction='out', length=6, width=1, colors='gray')


    # Create legend
    handles = [plt.Rectangle((0, 0), 1, 1, fc=colors[t]) for t in colors]
    ax.legend(handles, colors.keys(), loc='upper right')

    plt.tight_layout()
    return fig

# Main execution block for Colab
print("Please upload your Excel file now.")
print("The file should contain the following columns:")
print("'product name', 'quantity liters', 'process speed per hour', 'line efficiency', 'Change Over', 'Date from', 'Duration', 'Gap', 'First Wash Time'")
print("-" * 50)

uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Exiting.")
else:
    file_name = next(iter(uploaded))
    print(f"File '{file_name}' uploaded successfully.")

    try:
        # Read the Excel file into a DataFrame
        df = pd.read_excel(io.BytesIO(uploaded[file_name]))
        print("\nData loaded:")
        display(df.head())

        # Check for required columns
        required_columns = ['product name', 'quantity liters', 'process speed per hour', 'line efficiency', 'Change Over', 'Date from', 'Duration', 'Gap', 'First Wash Time']
        if not all(col in df.columns for col in required_columns):
            missing_cols = [col for col in required_columns if col not in df.columns]
            print(f"\nError: Missing required columns in the Excel file: {', '.join(missing_cols)}")
        else:
            # Generate the timeline plot
            fig = generate_timeline(df)

            if fig:
                output_filename = 'production_timeline.png'
                fig.savefig(output_filename, bbox_inches='tight')
                print(f"\nTimeline generated successfully. Downloading '{output_filename}'...")
                files.download(output_filename)

    except Exception as e:
        print(f"\nAn error occurred during processing: {e}")

ModuleNotFoundError: No module named 'google'